In [ ]:
# import necessary libraries

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import train_test_split

import time

print(tf.__version__)

2.3.0


In [ ]:
# define how many gpus are available and set a memmory limit
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ", len(gpus))
for i in range(len(gpus)):
    tf.config.experimental.set_virtual_device_configuration(gpus[i], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7900)]) 

Number of GPUs Available:  1


In [ ]:
strategy = tf.distribute.MirroredStrategy()
# the number of replicas that is created by the strategy should be equal to the number of GPU's available
print ('Number of synchronized replicas created: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of synchronized replicas created: 1


In [ ]:
# read in train and test data in case Google DRIVE is used
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Basepath for Google DRIVE:
Basepath = '/content/drive/My Drive/Stage_ENT_Studios_2/Data/Kaggle/Arrays_5GB_float32/'

# Basepath for Jupyter notebooks:
# Basepath = 'C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/Kaggle/Array_10GB/'
# also an array of 5GB and 20GB available

# Basepath for KILI
# Basepath = '/home/kili/Desktop/Data_FinalArrays/Kaggle/Arrays/'

# train data
train_images = np.load(Basepath + 'train_images_Final.npy')
print('Shape train images: {}'.format(train_images.shape))

train_labels =  np.load(Basepath + 'train_labels_Final.npy')
print('Shape train labels: {}'.format(train_labels.shape))

train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.33, random_state=42)

In [ ]:
# path to save the model and the tensorboard logs

# Basepath for Google DRIVE:
base_path = '/content/drive/My Drive/Stage_ENT_Studios_2/DR_Grading/Logs/'

# Basepath for jupyter notebooks:
# base_path = 'C:/Users/lunam/Documents/1steMaster/Stage/Code_Final/DR_classification/DeepLearningClassification/VGG16_GPU/Logs/'

# direction where the tensorboard files will be stored
log_dir_tens = base_path + 'Tensorboard_Logs/'
# direction where the trained models will be stored
log_dir_model = base_path + 'Trained_Model/'


In [ ]:
def VGG16(drop_prob = 0.1, init_filters = 64, dense_nodes = [4096,4096,1000]):
    # initialization of the weights
    W_init = tf.initializers.GlorotUniform()
    
    # input layer
    input_image = tf.keras.layers.Input(shape = (256,256,3))
        
    # Convolutional block 1
    conv2d_1 = tf.keras.layers.Conv2D(init_filters, (3, 3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(input_image)
    conv2d_2 = tf.keras.layers.Conv2D(init_filters, (3, 3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_1)
    pool_1 = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(conv2d_2)
    dropout_1 = tf.keras.layers.Dropout(rate = drop_prob)(pool_1)
        
    # Convolutional block 2
    conv2d_3 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_1)
    conv2d_4 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_3)
    pool_2 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_4)
    dropout_2 = tf.keras.layers.Dropout(rate = drop_prob)(pool_2 )
    
    # Convolutional block 3
    conv2d_5 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_2)
    conv2d_6 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_5)
    conv2d_7 = tf.keras.layers.Conv2D(4*init_filters, (1,1), activation= tf.nn.relu, kernel_initializer= W_init)(conv2d_6)
    pool_3 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_7)
    dropout_3 = tf.keras.layers.Dropout(rate = drop_prob)(pool_3)

    # Convolutional block 4
    conv2d_8 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_3)
    conv2d_9 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_8)
    conv2d_10 = tf.keras.layers.Conv2D(8*init_filters, (1,1), activation= tf.nn.relu, kernel_initializer= W_init)(conv2d_9)
    pool_4 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_10)
    dropout_4 = tf.keras.layers.Dropout(rate = drop_prob)(pool_4)

    # Convolutional block 5
    conv2d_11 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_4)
    conv2d_12 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_11)
    conv2d_13 = tf.keras.layers.Conv2D(8*init_filters, (1,1), activation= tf.nn.relu, kernel_initializer= W_init)(conv2d_12)
    pool_5 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_13)
    dropout_5 = tf.keras.layers.Dropout(rate = drop_prob)(pool_5)

    # fully connected layers
    flatten = tf.keras.layers.Flatten()(dropout_5)
    dense_1 = tf.keras.layers.Dense(dense_nodes[0], activation = tf.nn.relu, kernel_initializer= W_init)(flatten)
    dense_2 = tf.keras.layers.Dense(dense_nodes[1], activation = tf.nn.relu, kernel_initializer= W_init)(dense_1)
    dense_3 = tf.keras.layers.Dense(dense_nodes[2], activation = tf.nn.relu, kernel_initializer= W_init)(dense_2)
    output_label = tf.keras.layers.Dense(2, activation = tf.nn.softmax, kernel_initializer= W_init)(dense_3)


    model = tf.keras.Model(inputs=input_image, outputs=output_label)
    return model

In [ ]:
# loss function
def Bin_CrossEntropy_Loss(pred_labels, true_labels, GlobalBatchSize):

    true_labels_pos = true_labels[:,0]
    pred_labels_pos = pred_labels[:,0]

    loss_object = tf.keras.losses.BinaryCrossentropy(reduction= tf.keras.losses.Reduction.NONE)
    loss = loss_object([true_labels_pos], [pred_labels_pos])[0]
    return (loss/ GlobalBatchSize)

In [ ]:
def train_network(TrainImages, TrainLabels, TestImages, TestLabels, 
                  Drop_Prob = 0.1, Init_Filters = 64, Dense_Nodes = [4096,4096,1000], batch_size = 3, loss_function = 'BinCrossEntr', optim = 'Adam', 
                  learning_rate = tf.Variable(1e-5, dtype=tf.float32), MAX_EPOCH = 10, SaveResults = True, print_freq = 1):
    '''
    This function trains the UNet on the indicated train data with corresponding annotations
    At the end the trained model is being saved
    '''
    # setting up saver for the tensorboard logs
    if SaveResults:
        # creating summary which stores the results that can be visualised with tensorboard
        print("Setting up summary writer for tensorboard...")
        summary_writer = tf.summary.create_file_writer(log_dir_tens)

    # define the train and test batches that can be fed into the network
    # global batch size defines the batch size over all availabel GPU's
    print('Creating distributed data')
    Global_batch_size = batch_size * strategy.num_replicas_in_sync
    train_batch_data  = tf.data.Dataset.from_tensor_slices((TrainImages, TrainLabels)).shuffle(TrainImages.shape[0]).batch(Global_batch_size) 
    test_batch_data = tf.data.Dataset.from_tensor_slices((TestImages, TestLabels)).batch(Global_batch_size) 

    # distribute the data over the different GPU's
    train_dist_data =  strategy.experimental_distribute_dataset(train_batch_data)
    test_dist_data =  strategy.experimental_distribute_dataset(test_batch_data)

    # define the model that will be used for training and for testing
    # the model, optimisation and loss have to be distributed among GPU's
    tf.compat.v1.reset_default_graph()
    with strategy.scope():

        # model
        print('Defining the model')
        model = VGG16(drop_prob = Drop_Prob, init_filters = Init_Filters, dense_nodes = Dense_Nodes)
        
        # loss
        print('Defining loss')
        def compute_loss(PredictedLabels, TrueLabels):
            if loss_function == 'BinCrossEntr':
                loss = Bin_CrossEntropy_Loss(PredictedLabels, TrueLabels, Global_batch_size)
            return loss

        # optimization
        # a decaying learning rate is used
        steps_per_epoch = int(TrainImages.shape[0]/Global_batch_size)
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate= learning_rate, decay_steps= MAX_EPOCH*steps_per_epoch*0.25, decay_rate=0.2, staircase = True)
        print('Defining optimization')
        if optim == 'Adam':
            train_op = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
        elif optim == 'sgd':
            train_op = tf.keras.optimizers.SGD(learning_rate = lr_schedule)

        # defining the metrics
        print('Defining the metrics')
        train_roc_auc = tf.keras.metrics.AUC(curve = 'ROC')
        train_pr_auc = tf.keras.metrics.AUC(curve = 'PR')
        test_roc_auc = tf.keras.metrics.AUC(curve = 'ROC')
        test_pr_auc = tf.keras.metrics.AUC(curve = 'PR')

    # one train step is a step in which one batch of data is fed to every GPU
    # one train step is a step in which one batch of data is fed to every GPU
    def Train_Step(input):

        with tf.GradientTape() as tape:
            train_images_batch, train_labels_batch = input

            # make prediction with model
            pred_train_labels = model(train_images_batch, training = True)
            # compute loss
            train_err = compute_loss(pred_train_labels, train_labels_batch)

        # update model
        train_weights = model.trainable_variables
        gradients = tape.gradient(train_err, train_weights)
        train_op.apply_gradients(zip(gradients, train_weights))

        # compute auc and aupr score
        temp_train_labels = train_labels_batch[:,0]
        temp_train_pred_labels = pred_train_labels[:,0]
        train_roc_auc.update_state(temp_train_labels, temp_train_pred_labels)
        train_pr_auc.update_state(temp_train_labels, temp_train_pred_labels)

        # the error per replica is returned
        return train_err, train_roc_auc.result(), train_pr_auc.result()

    # for the last epoch some testing has to be done, in a test step one batch of test data is fed to every GPU
    def Test_Step(input):
        test_images_batch, test_labels_batch = input

        # make prediction with model
        pred_test_labels = model(test_images_batch, training = False)
        # compute loss
        test_err = compute_loss(pred_test_labels, test_labels_batch)
        
        # compute auc and aupr score
        temp_test_labels = test_labels_batch[:,0]
        temp_test_pred_labels = pred_test_labels[:,0]
        test_roc_auc.update_state(temp_test_labels, temp_test_pred_labels)
        test_pr_auc.update_state(temp_test_labels, temp_test_pred_labels)

        # the error per replica is returned
        return test_err, test_roc_auc.result(), test_pr_auc.result()


    @tf.function
    def distributed_train_step(dataset_inputs):
        per_replica_losses, per_replica_roc_auc, per_replica_pr_auc = strategy.run(Train_Step, args=(dataset_inputs,))
        return per_replica_losses, per_replica_roc_auc, per_replica_pr_auc

    @tf.function
    def distributed_test_step(dataset_inputs):
        per_replica_losses, per_replica_roc_auc, per_replica_pr_auc = strategy.run(Test_Step, args=(dataset_inputs,))
        return per_replica_losses, per_replica_roc_auc, per_replica_pr_auc
        
    
    # the train and test steps now have to be performed with the distributed strategy
    print('Training')
    for epo in range(1,MAX_EPOCH+1):
        start_time = time.time()
        
        n_train_steps = 0
        total_train_loss = 0
        total_train_auc = 0
        total_train_aupr = 0
        # go over all global batches
        for train_input_data in train_dist_data:
            n_train_steps+=1
            per_replica_train_losses, per_replica_train_roc_auc, per_replica_train_pr_auc = distributed_train_step(train_input_data)
            total_train_loss += strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_train_losses, axis=None)
            total_train_auc += strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_train_roc_auc, axis=None)
            total_train_aupr += strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_train_pr_auc, axis=None)

        # every print frequency the train and test resutls are printed out
        if epo % print_freq == 0 or epo == 1 or epo == (MAX_EPOCH):

            # calculate the final training results for this epoch
            total_train_loss = total_train_loss/n_train_steps
            total_train_auc = total_train_auc/n_train_steps
            total_train_aupr = total_train_aupr/n_train_steps

            # print out the train results
            print('epoch {} took {}s'.format(epo, time.time() - start_time))
            print('   train loss: {}'.format(total_train_loss))
            print('   train auc: {}'.format(total_train_auc))
            print('   train aupr: {}'.format(total_train_aupr))

            if SaveResults:      
                # save these values to visualize them later with tensorboard
                with summary_writer.as_default():
                    tf.summary.scalar('train_loss', total_train_loss, step = epo)
                    tf.summary.scalar('train_roc_auc', total_train_auc, step = epo)
                    tf.summary.scalar('train_pr_auc', total_train_aupr, step = epo)


            # some testing has to be done at these print frequencies
            print('Testing')
            n_test_steps = 0
            total_test_loss = 0
            total_test_auc = 0
            total_test_aupr = 0

            for test_input_data in test_dist_data:
                n_test_steps+=1
                per_replica_test_losses, per_replica_test_roc_auc, per_replica_test_pr_auc = distributed_test_step(test_input_data)
                total_test_loss += strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_test_losses, axis=None)
                total_test_auc += strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_test_roc_auc, axis=None)
                total_test_aupr += strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_test_pr_auc, axis=None)


            total_test_loss = total_test_loss/n_test_steps
            total_test_auc = total_test_auc/n_test_steps
            total_test_aupr = total_test_aupr/n_test_steps

            # print out the test results      
            print('   validation loss: {}'.format(total_test_loss))
            print('   validation auc: {}'.format(total_test_auc))
            print('   validation aupr: {}'.format(total_test_aupr))

            if SaveResults:
                with summary_writer.as_default():
                    tf.summary.scalar('validation_loss', total_test_loss, step = epo)
                    tf.summary.scalar('validation_roc_auc', total_test_auc, step = epo)
                    tf.summary.scalar('validation_pr_auc', total_test_aupr, step = epo)  
                    summary_writer.flush()     


        if SaveResults:
            # storing the model weights at two time-points
            if epo == int(MAX_EPOCH/2):
                print('Saving the intermediate model weights...')
                model.save_weights(log_dir_model + 'VGG16_' + str(epo) +'_epochs')
                print('Done')

            if epo == MAX_EPOCH:
                print('Saving the model weights...')
                model.save_weights(log_dir_model + 'VGG16_' + str(epo) +'_epochs')
                print('Done')

In [ ]:
train_network(train_images, train_labels, test_images, test_labels, MAX_EPOCH = 100, learning_rate = tf.Variable(1e-5, dtype=tf.float32), Drop_Prob = 0.3, batch_size = 40)
# (TrainImages, TrainLabels, TestImages, TestLables, 
#                   Drop_Prob = 0.1, Init_Filters = 64, Dense_Nodes = [4096,4096,1000], batch_size = 3, loss_function = 'BinCrossEntr', optim = 'Adam', 
#                   learning_rate = tf.Variable(1e-5, dtype=tf.float32), MAX_EPOCH = 10, SaveResults = True, print_freq = 1)

Setting up summary writer for tensorboard...
Creating distributed data
Defining the model
Defining loss
Defining optimization
Defining the metrics
Training
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/devi

KeyboardInterrupt: ignored

In [ ]:
print(test_labels)